In [9]:
import os
from os import mkdir
from genericpath import exists
from tqdm import tqdm
from time import sleep

import pandas as pd
import numpy as np
import tweepy
from textblob import TextBlob

from TwitterApi.tweet_api import TweetCrawlerAcademic


In [59]:
pd.set_option('display.float_format',  '{:,.0f}'.format)

In [60]:
# constant variables
START_TIME = "2021-01-01T00:00:00Z"
END_TIME = "2022-07-01T00:00:00Z"

In [61]:
client = TweetCrawlerAcademic("API.JSON", "crawler.log")

In [62]:
def set_output_name(string):
    name = string.replace(" ", "_")
    return name.replace(":", "")

In [63]:
def edit_dict_key_names(dictionary, dicname):
    l = dict()
    for key in dictionary.keys():
        newkey = f'{dicname}_{key}'
        l[newkey] = dictionary[key]
    return l

In [64]:
def retrieve_dataframes(store_path, page):
    users = pd.read_csv(f'{store_path}/{page}/users.csv')
    tweets = pd.read_csv(f'{store_path}/{page}/tweets.csv')
    references = pd.read_csv(f'{store_path}/{page}/refrences.csv', index_col='id')
    return users, tweets, references

In [65]:
query = "I apologize harassed is:quote -is:retweet lang:en"
filename = set_output_name(query)
print(filename)
store_path = f'data/{filename}'

I_apologize_harassed_isquote_-isretweet_langen


In [66]:
client.get_all_tweets(
    query=query,
    start_time=START_TIME,
    end_time=END_TIME,
    store_path=store_path
)

1it [00:08,  8.15s/it]


In [67]:
pd.read_csv(f'{store_path}/page1/tweets.csv')

,author_id,created_at,id,lang,public_metrics_like_count,public_metrics_quote_count,public_metrics_reply_count,public_metrics_retweet_count,quoted_id,replied_to_id,text
0,1365641149583622148,2022-06-19 21:43:13+00:00,1538638559455977472,en,3,0,0,0,"1,538,555,301,502,369,792",NaN,Remember when I was harassed for dissing this ...
1,232148063,2022-06-18 21:51:05+00:00,1538278148361576449,en,43,0,0,8,"1,537,818,490,366,746,624",NaN,"""I apologize to every woman in the country,on ..."
2,222371467,2022-06-17 12:41:00+00:00,1537777328448626688,en,0,0,0,1,"1,537,449,199,888,871,424",NaN,Video of Ishin candidates from its official si...
3,794583663040049152,2022-06-16 18:59:33+00:00,1537510204694069248,en,5,0,1,0,"1,537,156,937,212,645,376",NaN,FUCK WALGREENS. I was 16 when my manager sexua...
4,222371467,2022-06-16 18:08:19+00:00,1537497313135841281,en,0,0,0,1,"1,537,449,199,888,871,424",NaN,"Candidates from Ishin\n\nInose: ""I would like ..."
...,...,...,...,...,...,...,...,...,...,...,...
186,1109505319103717376,2021-01-19 22:34:03+00:00,1351659214242893825,en,0,0,1,0,"1,351,658,501,924,286,464","1,351,658,169,056,034,816",@amourvms i explained this confusing as hell b...
187,1287212544327741440,2021-01-19 19:54:40+00:00,1351619103690117121,en,0,0,0,0,"1,351,571,278,478,471,168",NaN,IT TOOK U A FVKING YEAR TO APOLOGIZE!?He got c...
188,1327264018130722818,2021-01-16 03:36:55+00:00,1350285878829191176,en,1,0,0,0,NaN,NaN,"I was a nanny one summer for 3 boys aged 12, 1..."
189,1180356114287878144,2021-01-09 15:17:03+00:00,1347925359635927041,en,4,0,2,0,NaN,NaN,no bc y’all let that translation acc start thi...


In [68]:
def get_all_replies(client, path, conversation_id):
    directory = f'{path}/{conversation_id}'
    if not os.path.exists(directory):
        mkdir(directory)
    conv_query = f'conversation_id:{conversation_id} -is:retweet'
    
    out = pd.DataFrame()
    for i, respond in tqdm(enumerate(tweepy.Paginator(client.client.search_all_tweets,
                                                      query=conv_query,
                                                      max_results=500,
                                                      start_time=START_TIME,
                                                      end_time=END_TIME))):
        if respond.data is None:
            print("respond.data is none")
            continue
        out = out.append(pd.DataFrame(client.__get_respond_dfs__(respond)['tweets']), ignore_index=True)
        sleep(1)
    out.to_excel(f'{directory}/replies.xlsx', index=False)
    print(f'File saved at {directory}/replies.txt')
    return out                     

In [69]:
get_all_replies(client, 'data', 1505934049877012483)

1it [00:01,  1.45s/it]

File saved at data/1505934049877012483/replies.txt


,id,text
0,1505943269984907264,@arekushi14 @zYuko13 do it for the love of god...
1,1505942097454587907,@arekushi14 WHAT THE FUCK
2,1505940771697881088,@chocomint_twt she’s gonna say she’s gonna cha...
3,1505940725770293248,@INFAMY_DT it dont matter to me anymore dawg i...
4,1505940534233341956,@arekushi14 Lexy oh my god I hope you’re ok
5,1505940207740153862,@chocomint_twt well it’s whatever now. just do...
6,1505940097866338309,@arekushi14 @GMDSepiuh also wtf
7,1505940051825500162,@arekushi14 @GMDSepiuh HOW DID YOU REPLY IN LI...
8,1505939945973862403,@aumsoftreal @arekushi14 @GMDSepiuh 17 LOL
9,1505939941120872448,@aumsoftreal @GMDSepiuh 17


In [70]:
# def get_sq_tweets_of_page(store_path, page):    

#     self_quote_tweets = pd.DataFrame()
#     _, tweets, references = retrieve_dataframes(store_path, page)
#     for t_id, q_id in tweets[['author_id', 'quoted_id']].values:
#         quoted_tweets = references.loc[q_id]
#         if len(quoted_tweets.index) == 0:
#             continue
#         quoted_tweet = quoted_tweets.iloc[0]
#         if (quoted_tweet.author_id == t_id):
#             dict1 = edit_dict_key_names(dict(tweets.iloc[tweet]), 'tweets')
#             dict2 = edit_dict_key_names(dict(quoted_tweet), 'references')
#             merged_dict = {**dict1, **dict2}
#             self_quote_tweets = self_quote_tweets.append(merged_dict, ignore_index=True)

#     print(self_quote_tweets.shape)
#     return self_quote_tweets



def get_sq_tweets_of_page(store_path, page):    

    # self_quote_tweets = pd.DataFrame()
    _, tweets, references = retrieve_dataframes(store_path, page)
    tweets.dropna(subset=['quoted_id'], inplace=True)
    # print(tweets)
    df = pd.merge(
        tweets[['id', 'author_id', 'quoted_id']],
        references[['author_id']],
        left_on='quoted_id',
        right_index=True,
        suffixes=('_original', '_refrence')
    )
    return df, df[df['author_id_original'] == df['author_id_refrence']].sort_values('id')

In [71]:
total, result = get_sq_tweets_of_page(store_path, 'page1')
result

,id,author_id_original,quoted_id,author_id_refrence
35,1512883025234718720,1378503060893278212,"1,512,651,545,271,103,488",1378503060893278212


In [72]:
total.sort_values('id')

,id,author_id_original,quoted_id,author_id_refrence
185,1352998385876422662,232084968,"1,352,817,037,589,045,248",347652108
184,1353432163522383873,289595909,"1,353,410,181,011,447,808",654203
180,1355164178525216769,21513572,"1,355,025,490,982,133,760",2884771
170,1366846258124623880,2409456320,"1,366,155,745,398,575,104",19367849
166,1369516032969252872,832206448830590977,"1,369,118,773,446,328,320",17659957
157,1375879419554902022,1348029591982649346,"1,375,760,101,013,823,488",1355764138958610434
138,1389355272179576834,1150818840864931840,"1,389,232,124,599,824,384",1352837694565404677
136,1390825038253854720,333742261,"1,390,745,370,167,185,408",1295872862297890816
120,1405655581277437956,255340030,"1,405,654,039,174,422,528",1390787888
113,1416655326946873344,403835551,"1,415,395,245,177,532,416",17372405


In [73]:
_, t, r = retrieve_dataframes(store_path, 'page1')

In [74]:
t.sort_values('quoted_id').iloc[100:]

,author_id,created_at,id,lang,public_metrics_like_count,public_metrics_quote_count,public_metrics_reply_count,public_metrics_retweet_count,quoted_id,replied_to_id,text
41,2719830224,2022-03-20 17:45:21+00:00,1505601401602330628,en,13,0,2,3,"1,505,470,189,818,859,520",NaN,I fully admit to living 75% of my life in my h...
40,1349196499708596225,2022-03-21 16:05:19+00:00,1505938617054556163,en,74,0,13,0,"1,505,934,049,877,012,480",NaN,want to apologize for putting this shit on you...
39,1322714127270207488,2022-03-30 07:04:32+00:00,1509064012041912320,en,0,0,0,0,"1,508,987,165,635,940,352",NaN,I mean they are one of those who harassed hér ...
37,994875023490985985,2022-03-30 11:51:21+00:00,1509136194726072320,en,0,0,0,0,"1,509,065,522,448,519,168",NaN,Harassed? Where? All I can see is your co-bree...
13,985323765277057025,2022-06-05 10:24:36+00:00,1533394350050316288,en,5,0,2,1,"1,509,738,311,614,537,728",NaN,hi hi! I'm not strictly nblw or nblm so I won'...
...,...,...,...,...,...,...,...,...,...,...,...
173,969804583315755008,2021-02-13 18:11:06+00:00,1360652735427694594,en,0,0,0,0,NaN,NaN,((I understand representation matters; I know ...
183,1135472562358673408,2021-01-25 15:47:10+00:00,1353731142587977732,en,2,0,1,0,NaN,NaN,If i have ever harassed you in the DM. I am so...
188,1327264018130722818,2021-01-16 03:36:55+00:00,1350285878829191176,en,1,0,0,0,NaN,NaN,"I was a nanny one summer for 3 boys aged 12, 1..."
189,1180356114287878144,2021-01-09 15:17:03+00:00,1347925359635927041,en,4,0,2,0,NaN,NaN,no bc y’all let that translation acc start thi...


In [ ]:
for page in tqdm(os.listdir(store_path)):
    self_quote_tweets = get_sq_tweets_of_page(store_path, page)
    
    sq_path = f'{store_path}/{page}'
    self_quote_tweets[1].astype('str').to_csv(f'{sq_path}/self_quote_tweets.csv', index=False)
    print(f'Extracted sq tweets of {page}')
    
    conversation_ids = self_quote_tweets['references_id'].tolist()
    
    for conv_id in conversation_ids:
        print(int(conv_id))
        continue
        reps = get_all_replies(client, sq_path, int(conv_id))
        print(f'Retrieved replies of {int(conv_id)} with shape {reps.shape}')
        sleep(2)

## Sentiment Analysis

In [97]:
replies = pd.read_excel('data/I_apologize_harassed_isquote_-isretweet_langen/page1/1512651545271103488/replies.xlsx')
replies.shape

(13, 2)

In [98]:
pd.set_option('display.float_format',  '{:,.2f}'.format)

In [99]:
replies = replies.assign(sentiment=None)
sentences = replies['text'].tolist()
for i, sentence in enumerate(sentences):
    res = TextBlob(sentence)
    mood = res.sentiment.polarity
    replies.loc[replies['text'] == sentence, 'sentiment'] = mood

In [102]:
replies.to_csv('testtt.csv')